## Ejemplo análisis de una sección

In [ ]:
from mento import Concrete_ACI_318_19, SteelBar, RectangularBeam
from mento import cm, MPa

conc= Concrete_ACI_318_19(name="C25",f_c=25*MPa) 
steel= SteelBar(name="ADN 420", f_y=420*MPa) 
beam = RectangularBeam(label="101",concrete=conc,steel_bar=steel,width=20*cm, height=40*cm)
beam.data

In [ ]:
from mento import Concrete_ACI_318_19, SteelBar, Forces, RectangularBeam, Node
from mento import mm, cm, kN, MPa
# Defines materiales and beam section
conc= Concrete_ACI_318_19(name="C25",f_c=25*MPa) 
steel= SteelBar(name="ADN 420", f_y=420*MPa) 
beam = RectangularBeam(label="101",concrete=conc,steel_bar=steel,width=20*cm, height=40*cm)
# Define forces
f1 = Forces(label='1.4D', V_z=50*kN)
f2 = Forces(label='1.2D+1.6L', V_z=155*kN)
f3 = Forces(label='W', V_z=220*kN)
f4 = Forces(label='S', V_z=80*kN)
f5 = Forces(label='E', V_z=10*kN)
# Create node and assign beam section and list of forces
Node(section=beam, forces_list=[f1, f2, f3, f4, f5])
# Assign transverse reiforcement
beam.set_transverse_rebar(n_stirrups=1, d_b=8*mm, s_l=25*cm)
# Check shear for all forces
beam.check_shear()
# print(beam.check_shear())
beam.results
# beam.design_shear()
# beam.results
# beam.shear_results_detailed()

Beam 101, $b$=20.00 cm, $h$=40.00 cm, $c_{c}$=2.50 cm,                             Concrete C25, Rebar ADN 420.

Shear reinforcing 1eØ8/25 cm, $A_v$=4.02 cm²/m, $V_u$=220 kN, $\phi V_n$=89.08 kN → $\color{#d43e36}{\text{DCR}=2.47}$

In [3]:
from mento.summary import BeamSummary
import pandas as pd

conc = Concrete_ACI_318_19(name="C25", f_c=25*MPa)
steel = SteelBar(name="ADN 420", f_y=420*MPa)
input_df = pd.read_excel('Mento-Input.xlsx', sheet_name='Beams', usecols='B:R', skiprows=4)
# data = {'Label': ['', 'V101', 'V102', 'V103', 'V104'],
#         'b': ['cm', 20, 20, 20, 20],
#         'h': ['cm', 50, 50, 50, 50],
#         'Nx': ['kN', 0, 0, 0, 50],
#         'Vz': ['kN', 20, 50, 100, 100],
#         'My': ['kNm', 0, 35, 40, 45],
#         'ns': ['', 0, 1.0, 1.0, 1.0],
#         'dbs': ['mm', 0, 6, 6, 6],
#         'sl': ['cm', 0, 20, 20, 20],
#         'n1': ['', 2.0, 2.0, 2.0, 2.0],
#         'db1': ['mm', 12, 12, 12, 12],
#         'n2': ['', 1.0, 0.0, 1.0, 0.0],
#         'db2': ['mm', 10, 0, 10, 0],
#         'n3': ['', 2.0, 0.0, 2.0, 0.0],
#         'db3': ['mm', 12, 0, 0, 0],
#         'n4': ['', 1.0, 0.0, 1.0, 0.0],
#         'db4': ['mm', 10, 0, 0, 0]}
# input_df = pd.DataFrame(data)
# print(input_df)
beam_summary = BeamSummary(concrete=conc, steel_bar=steel, beam_list=input_df)
# beam_summary.data
# print(beam_summary.data)
# beam_summary.capacity()
# beam_summary.check()
# beam_summary.check().to_excel('hola.xlsx', index=False)
beam_summary.shear_results(capacity=False)
# beam_summary.shear_results(capacity=True)
# print(results)
# print(beam_summary.beams[3]['section'].shear_results_detailed)

ValueError: No Node or forces list associated with this beam.